**Thol EoS for Lennard Jones fluid**

In [8]:
from eos_core.user_defined import *
from eos_core.user_defined.num_dual import *
from eos_core import *
from eos_core.si import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
# Parameters for Thol EoS:
A = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0,
              2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])
N = np.array([0.005208073, 2.186252000, -2.161016000, 1.452700000, -2.041792000, 0.186952860, -0.090988445, 
              -0.497456100, 0.109014310, -0.800559220, -0.568839000, -0.620862500, -1.466717700, 1.891469000, 
              -0.138370100, -0.386964500, 0.126570200, 0.605781000, 1.179189000, -0.477326790, -9.921857500, -0.574793200, 0.003772923])
T = np.array([1.000, 0.320, 0.505, 0.672, 0.843, 0.898, 1.294, 2.590, 1.786, 2.770, 1.786,
              1.205, 2.830, 2.548, 4.650, 1.385, 1.460, 1.351, 0.660, 1.496, 1.830, 1.616, 4.970])
D = np.array([4.0, 1.0, 1.0, 2.0, 2.0, 3.0, 5.0, 2.0, 2.0, 3.0, 1.0,
              1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 2.0, 1.0, 2.0, 3.0, 1.0, 1.0])
L = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 1.0, 2.0, 2.0,
              1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
ETA = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.067, 1.522,
                8.82, 1.722, 0.679, 1.883, 3.925, 2.461, 28.2, 0.753, 0.82])
BETA = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.625,
                 0.638, 3.91, 0.156, 0.157, 0.153, 1.16, 1.73, 383, 0.112, 0.119])
GAMMA = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.71,
                  0.86, 1.94, 1.48, 1.49, 1.945, 3.02, 1.11, 1.17, 1.33, 0.24])
EPSILON = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2053,
                    0.409, 0.6, 1.203, 1.829, 1.397, 1.39, 0.539, 0.934, 2.369, 2.43])

In [75]:
def zero(state):
    if isinstance(state, StateF):
        return 0.0
    elif isinstance(state, StateD):
        return Dual64.from_re(0.0)
    elif isinstance(state, StateHD):
        return HyperDual64.from_re(0.0)
    elif isinstance(state, StateHD3):
        return HD3_64.from_re(0.0)
    elif isinstance(state, StateHD3D):
        return HD3Dual64.from_re(Dual64.from_re(0.0))
    else:
        HyperDualDual64.from_re(Dual64.from_re(0.0))
        
class Thol:
    def __init__(self):
        self.sigma = 3.7039 * ANGSTROM
        self.eps_k = 150.03 * KELVIN
        self.tc = 1.32 * self.eps_k / KELVIN
        self.rhoc = 0.31 / self.sigma**3 * ANGSTROM**3
        
    def components(self): 
        return 1
    
    def get_subset(self, components):
        return self
        
    def molar_weight(self):
        return 1.0
    
    def max_density(self, moles):
        return 0.05
    
    def helmholtz_energy(self, state):
        """
        state (StateHD):
            temperature in Kelvin als Float, Dual oder HD oder HD3, HDD, HDD3,
            partial_density in # / Angstrom^3
            volume in Angstrom^3
            moles in mol
        """
        tau = self.tc / state.temperature
        delta = np.sum(state.partial_density) / self.rhoc
        a = 0.0 
        for i in range(6):
            a = a + N[i] * delta**D[i] * tau**T[i]
        for i in range(6, 12):
            a = a + N[i] * delta**D[i] * tau**T[i] * np.exp(-1.0 * delta**L[i])
        for i in range(12, 23):
            a = a + N[i] * delta**D[i] * tau**T[i] * np.exp(- 1.0 * ETA[i] * (delta - EPSILON[i])**2.0 - 1.0 * BETA[i] * (tau - 1.0 * GAMMA[i])**2.0)
        return a * np.sum(state.moles)

In [76]:
# Test Tol EoS
thol = Thol()
eos = UserDefinedEos(thol)
temperature = 293 * KELVIN
density = 0.4 / (thol.sigma**3 * NAV) 
state1 = State(eos, temperature=temperature, density=density)

In [77]:
150.03 * 1.337

200.59011

In [78]:
State(eos, 200*KELVIN, pressure=BAR)

|temperature|density|
|-|-|
|200 K|60.4902481804112  mol/m³|

**Test Thol EoS**

In [82]:
cp = State.critical_point(eos, initial_temperature=100*KELVIN, verbosity=Verbosity.Iter)

 iter |    residual    |   temperature   |       density        
----------------------------------------------------------------
    0 |                | 100 K | 24.908086007607697 kmol/m³
    1 |   8.39127369e1 | 125 K | 23.46547454036538 kmol/m³
    2 |   5.72497912e1 | 156.25 K | 21.368223826627954 kmol/m³
    3 |   3.01554839e1 | 181.05344088134393 K | 18.877415225867182 kmol/m³
    4 |   1.32905051e1 | 196.32751112519156 K | 16.386606625106413 kmol/m³
    5 |   5.67030703e0 | 206.01853855630475 K | 13.895798024345645 kmol/m³
    6 |   1.84070432e0 | 201.8677241721009 K | 12.267793599231378 kmol/m³
    7 |  5.73606688e-1 | 199.99382239196314 K | 11.051174021799556 kmol/m³
    8 |  1.46084010e-1 | 198.61792362282236 K | 10.525858279018173 kmol/m³
    9 |  2.52852860e-2 | 198.10930515257152 K | 10.404705661894072 kmol/m³
   10 |  3.49064040e-3 | 198.0425480080979 K | 10.339662762669684 kmol/m³
   11 |  6.40627917e-4 | 198.0398300789862 K | 10.28288036150796 kmol/m³
   12 |  1.638614

In [83]:
cp.temperature / thol.eps_k

1.3200003469821544

In [85]:
vle = VLEState.pure_p(eos, 1.0 * BAR)
vle

||temperature|density|
|-|-|-|
|**vapor**|111.67250977757882 K|111.05992974448719  mol/m³|
|**liquid**|111.67250977757882 K|26.91278587289406 kmol/m³|

In [125]:
state = State(eos, temperature=300.0 * KELVIN, pressure=1.0 * BAR)

In [109]:
vle = VLEState.new_pure_p(eos, 1.0*BAR)

PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('not implemented!'), traceback: Some(<traceback object at 0x7f391548bc00>) }

In [40]:
vle = VLEState.boiling_temperature(eos, 1.0 * BAR)

PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('get_subset() takes 1 positional argument but 2 were given'), traceback: None }

In [53]:
vle = VLEState.bubble_point_px(eos, 5.0 * BAR, np.array([1.0]))

<ipython-input-3-5d7812ec67f3>:32: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(0.5)


RuntimeError: Invalid state. Temperature is not finite.

<ipython-input-3-5d7812ec67f3>:32: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(0.5)


PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('not implemented!'), traceback: Some(<traceback object at 0x7f39157aed00>) }

In [61]:
vle = VLEState.vle_pure_comps_p(eos, 1.0*BAR)

PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('get_subset() takes 1 positional argument but 2 were given'), traceback: None }

In [63]:
vle = VLEState.vle_pure_comps_t(eos, 300*KELVIN)

PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('get_subset() takes 1 positional argument but 2 were given'), traceback: None }

In [11]:
vle = VLEState.
#state1.critical_point_pure(eos)

AttributeError: type object 'builtins.VLEState' has no attribute 'pure_p'

In [ ]:
#VLEState.new_pure_t(eos,temperature)

In [12]:
state = State(eos, temperature=300.0 * KELVIN, pressure=1.0 * BAR)

<ipython-input-3-5d7812ec67f3>:32: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(0.5)


In [17]:
vle = VLEState.new_pure_p(eos, 1.0 * BAR )
#
#pure_p(eos, 1.0 * BAR)

<ipython-input-3-5d7812ec67f3>:32: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(0.5)


PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('not implemented!'), traceback: Some(<traceback object at 0x7f391591f6c0>) }

In [29]:
vle = VLEState.vle_pure_comps_t(eos, 100.0*KELVIN)

PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('get_subset() takes 1 positional argument but 2 were given'), traceback: None }

In [20]:
vle

<function VLEState.vle_pure_comps_t(eos, temperature)>